In [1]:
import ipaddress
import logging
from datetime import date, datetime

import pandas as pd
from dacite import from_dict
from metar import Metar
from shapely import Polygon

from aimlsse_api.client import GroundDataClient
from aimlsse_api.data.metar import *

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
ground_data_client = GroundDataClient(ipaddress.ip_address('127.0.0.1'), 8000)

In [4]:
properties = [
    MetarProperty(MetarPropertyType.WIND_SPEED, UnitSpeed.KILOMETERS_PER_HOUR),
    MetarProperty(MetarPropertyType.TIME),
    MetarProperty(MetarPropertyType.CURRENT_WEATHER),
    MetarProperty(MetarPropertyType.RUNWAY_VISIBILITY, UnitDistance.METERS),
    MetarProperty(MetarPropertyType.RECENT_WEATHER),
    MetarProperty(MetarPropertyType.SKY_CONDITIONS, UnitDistance.METERS),
    MetarProperty(MetarPropertyType.RUNWAY_WINDSHEAR),
    MetarProperty(MetarPropertyType.METAR_CODE),
    MetarProperty(MetarPropertyType.OBSERVATION_CYCLE),
    MetarProperty(MetarPropertyType.VISIBILITY, UnitDistance.METERS),
    MetarProperty(MetarPropertyType.VISIBILITY_DIRECTION)
]
property_names = [str(property) for property in properties]
print(property_names)

['wind_speed [KMH]', 'time', 'current_weather', 'runway_visibility [M]', 'recent_weather', 'sky_conditions [M]', 'runway_windshear', 'metar_code', 'observation_cycle', 'visibility [M]', 'visibility_direction']


In [5]:
data = ground_data_client.queryMetar(date(2022, 8, 1), date(2022, 8, 6), properties,
    stations=['EDDE','EDDF']
)

In [6]:
data = ground_data_client.queryMetar(date(2022, 8, 1), date(2022, 8, 6), properties,
    polygons=[Polygon([(6.0, 47.5),(15.0, 47.5),(15.0, 55.0),(6.0, 55.0)])]
)

In [7]:
print(data.columns)
print(data.dtypes)

Index(['station', 'datetime', 'wind_speed [KMH]', 'time', 'current_weather',
       'runway_visibility [M]', 'recent_weather', 'sky_conditions [M]',
       'runway_windshear', 'metar_code', 'observation_cycle', 'visibility [M]',
       'visibility_direction'],
      dtype='object')
station                          object
datetime                 datetime64[ns]
wind_speed [KMH]                float64
time                     datetime64[ns]
current_weather                  object
runway_visibility [M]            object
recent_weather                   object
sky_conditions [M]               object
runway_windshear                 object
metar_code                       object
observation_cycle                 int32
visibility [M]                  float64
visibility_direction            float64
dtype: object


In [8]:
print(data[property_names])

        wind_speed [KMH]                time  \
0              11.111990 2022-08-01 00:25:00   
1               7.407994 2022-08-01 00:55:00   
2               9.259992 2022-08-01 01:25:00   
3               9.259992 2022-08-01 01:55:00   
4               7.407994 2022-08-01 02:25:00   
...                  ...                 ...   
432295          7.407994 2022-11-30 21:50:00   
432296          7.407994 2022-11-30 22:20:00   
432297          7.407994 2022-11-30 22:50:00   
432298          5.555995 2022-11-30 23:20:00   
432299          5.555995 2022-11-30 23:50:00   

                                          current_weather  \
0       [DataWeather(intensity='', description=None, p...   
1       [DataWeather(intensity='', description=None, p...   
2       [DataWeather(intensity='', description=None, p...   
3       [DataWeather(intensity='', description=None, p...   
4       [DataWeather(intensity='', description=None, p...   
...                                                   ...

In [9]:
print(data['current_weather'][0])
print(data['recent_weather'][0])
print(data['runway_visibility [M]'][0])
print(data['runway_windshear'][0])
print(data['sky_conditions [M]'][0])

[DataWeather(intensity='', description=None, precipitation='///', obscuration=None, other=None), DataWeather(intensity='', description=None, precipitation='/////', obscuration=None, other=None)]
[]
[]
[]
[]


In [10]:
runway_data = data['runway_visibility [M]']
print(runway_data)

0         []
1         []
2         []
3         []
4         []
          ..
432295    []
432296    []
432297    []
432298    []
432299    []
Name: runway_visibility [M], Length: 400576, dtype: object


In [11]:
metar_code = data['metar_code'].iloc[0]
print(metar_code)

EBLB 010025Z AUTO 27006KT //// /// ///// Q////


This way, the following properties are made available:
```
[
    'station_id', 'time', 'cycle', 'type', 'mod', 'visibility_m',
    'temperature_C', 'dew_point_C', 'wind_speed_kmh', 'wind_dir', 
    'pressure_mb', 'sea_level_pressure_mb', 'snowdepth_m', 'code'
]
```
Any additional flags must be computed separately, like:
```
[
    'lowcloud_cover', 'lowcloud_height_m', 'lowcloud_type', 'CAVOK'
]
```

In [12]:
data['CAVOK'] = data['metar_code'].str.contains('CAVOK')
print(data['CAVOK'])

0         False
1         False
2         False
3         False
4         False
          ...  
432295    False
432296    False
432297    False
432298    False
432299    False
Name: CAVOK, Length: 400576, dtype: bool
